In [1]:
import numpy as np
import pandas as pd
from my_functions import *
import datetime as dt

----
## Tabla Objetivo
El objetivo es crear una tabla principal que tenga la siguiente forma:

In [2]:
pd.DataFrame(columns=['fecha', 'dia_semana', 'semana', 'dolar_b', 'dolar_o', 'brecha_s', 'brecha','db_var','do_var','brecha_var', 'evento_p', 'evento_t'])

,fecha,dia_semana,semana,dolar_b,dolar_o,brecha_s,brecha,db_var,do_var,brecha_var,evento_p,evento_t


Explicación de cada columna:


Columna         | Explicación                                                       | Tipo de dato  |
----------------|-------------------------------------------------------------------|---------------|
**fecha**       |Fecha en formato YYYY-mm-dd                                        | Datetime      |
**dia_semana**  |Día de la semana en inglés correspondiente a la fecha              | String        | 
**semana**      |* Año y número de semana en el año en formato YYYY-WW              | String        | 
**dolar_b**     |Valor en soles del dolar blue: $Db_i$                              | Flotante      |
**dolar_o**     |Valor en soles del dolar oficial: $Do_i$                           | Flotante      |
**brecha_s**    |Brecha en soles entre los dólares: $Bs_i = Db_i - Do_i$            | Flotante      |
**brecha**      |Brecha en porcentaje entre los dólares: $B_i = (Db_i/Do_i-1)*100$  | Flotante      |
**db_var**      |Variación de dolar blue: $Vdb_i = (Db_i / Db_{i-1}-1)*100$         | Flotante      |
**do_var**      |Variación de dolar oficial: $Vdo_i = (Do_i / Do_{i-1}-1)*100$      | Flotante      |
**brecha_var**  |** Variación de la brecha: $Vbr_i = B_i-B_{i-1}$                   | Flotante      |
**evento_p**    |Personaje histórico correspondiente al evento                      | String        |
**evento_t**    |Tipo de evento correspondiente                                     | String        |

**Los días que preceden el primer lunes son considerados en la semana 0* <br>
** *La variación de la brecha no usa porcentajes puesto que la brecha misma representa porcentaje*<br>

----
## Construyendo la tabla.
Primero realizo un arreglo con el rango de fechas desde el 2002-03-04 hasta ayer

In [3]:
# Fechas
primer_dia = '2002-03-04'
ayer = dt.datetime.strftime(dt.datetime.today() - dt.timedelta(days=1), '%Y-%m-%d')
table = pd.DataFrame(pd.date_range(start=primer_dia, end=ayer), columns=['fecha'])

# Día de la semana:
table['dia_semana'] = table['fecha'].map(lambda x: dt.datetime.strftime(x,'%A'))

# Semana
table['semana'] = table['fecha'].map(lambda x: dt.datetime.strftime(x,'%Y-%W'))
table.head(10)

,fecha,dia_semana,semana
0,2002-03-04,Monday,2002-09
1,2002-03-05,Tuesday,2002-09
2,2002-03-06,Wednesday,2002-09
3,2002-03-07,Thursday,2002-09
4,2002-03-08,Friday,2002-09
5,2002-03-09,Saturday,2002-09
6,2002-03-10,Sunday,2002-09
7,2002-03-11,Monday,2002-10
8,2002-03-12,Tuesday,2002-10
9,2002-03-13,Wednesday,2002-10


In [4]:
table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7459 entries, 0 to 7458
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   fecha       7459 non-null   datetime64[ns]
 1   dia_semana  7459 non-null   object        
 2   semana      7459 non-null   object        
dtypes: datetime64[ns](1), object(2)
memory usage: 174.9+ KB


Importo las tablas relacionadas con el dolar a la tabla principal:

In [5]:
# Leeo tablas
dolar_b = pd.read_csv('API_tables/usd_b.csv')
dolar_o = pd.read_csv('API_tables/usd_o.csv')
eventos = pd.read_csv('API_tables/events.csv')

In [6]:
# Modifico el tipo de dato en 'd' para que se facilite unir las tablas con la tabla principal
dolar_b['d'] = pd.to_datetime(dolar_b['d'])
dolar_o['d'] = pd.to_datetime(dolar_o['d'])
eventos['d'] = pd.to_datetime(eventos['d'])

# Uno las tablas correspondientes al dolar
table = table.join(dolar_b.set_index('d'), on='fecha', how='left').rename(columns={'v': 'dolar_b'})
table = table.join(dolar_o.set_index('d'), on='fecha', how='left').rename(columns={'v': 'dolar_o'})

table.head(10)

,fecha,dia_semana,semana,dolar_b,dolar_o
0,2002-03-04,Monday,2002-09,2.12,2.01
1,2002-03-05,Tuesday,2002-09,2.12,1.99
2,2002-03-06,Wednesday,2002-09,2.14,2.05
3,2002-03-07,Thursday,2002-09,2.22,2.14
4,2002-03-08,Friday,2002-09,2.27,2.20
5,2002-03-09,Saturday,2002-09,NaN,NaN
6,2002-03-10,Sunday,2002-09,NaN,NaN
7,2002-03-11,Monday,2002-10,2.32,2.25
8,2002-03-12,Tuesday,2002-10,2.43,2.37
9,2002-03-13,Wednesday,2002-10,2.42,2.33


Para los valores de **los dólares**, rellenaré los datos faltantes con valores numéricos que se encuentren entre los valores que lo rodean y permitan ver un crecimiento lineal. <br>
Ejemplo: <br>
Con Datos Vacíos    | Rellenando datos vacíos   |
--------------------|---------------------------|
3                   |3                          |
NaN                 |**3.666666**               |
Nan                 |**4.333333**               |
5                   |5                          |

En este ejemplo se rellenó usando 0.666 como diferencia necesaria para crear datos con crecimiento lineal.

In [7]:
# Rellenamos
table['dolar_b'] = table['dolar_b'].interpolate()
table['dolar_o'] = table['dolar_o'].interpolate()
table.head(10)

,fecha,dia_semana,semana,dolar_b,dolar_o
0,2002-03-04,Monday,2002-09,2.120000,2.010000
1,2002-03-05,Tuesday,2002-09,2.120000,1.990000
2,2002-03-06,Wednesday,2002-09,2.140000,2.050000
3,2002-03-07,Thursday,2002-09,2.220000,2.140000
4,2002-03-08,Friday,2002-09,2.270000,2.200000
5,2002-03-09,Saturday,2002-09,2.286667,2.216667
6,2002-03-10,Sunday,2002-09,2.303333,2.233333
7,2002-03-11,Monday,2002-10,2.320000,2.250000
8,2002-03-12,Tuesday,2002-10,2.430000,2.370000
9,2002-03-13,Wednesday,2002-10,2.420000,2.330000


Calculo las **brechas**

In [8]:
table['brecha_s'] = table['dolar_b'] - table['dolar_o']
table['brecha'] = (table['dolar_b']/table['dolar_o'] - 1)*100
table.head(10) 

,fecha,dia_semana,semana,dolar_b,dolar_o,brecha_s,brecha
0,2002-03-04,Monday,2002-09,2.120000,2.010000,0.11,5.472637
1,2002-03-05,Tuesday,2002-09,2.120000,1.990000,0.13,6.532663
2,2002-03-06,Wednesday,2002-09,2.140000,2.050000,0.09,4.390244
3,2002-03-07,Thursday,2002-09,2.220000,2.140000,0.08,3.738318
4,2002-03-08,Friday,2002-09,2.270000,2.200000,0.07,3.181818
5,2002-03-09,Saturday,2002-09,2.286667,2.216667,0.07,3.157895
6,2002-03-10,Sunday,2002-09,2.303333,2.233333,0.07,3.134328
7,2002-03-11,Monday,2002-10,2.320000,2.250000,0.07,3.111111
8,2002-03-12,Tuesday,2002-10,2.430000,2.370000,0.06,2.531646
9,2002-03-13,Wednesday,2002-10,2.420000,2.330000,0.09,3.862661


Calculo ahora la **variación** de cada dólar

In [9]:
table = add_variation(table, ['dolar_b', 'dolar_o'])
table.rename(columns={'dolar_b_var': 'db_var','dolar_o_var': 'do_var'}, inplace = True)
table

,fecha,dia_semana,semana,dolar_b,dolar_o,brecha_s,brecha,db_var,do_var
0,2002-03-04,Monday,2002-09,2.120000,2.010000,0.110000,5.472637,0.000000,0.000000
1,2002-03-05,Tuesday,2002-09,2.120000,1.990000,0.130000,6.532663,0.000000,-0.995025
2,2002-03-06,Wednesday,2002-09,2.140000,2.050000,0.090000,4.390244,0.943396,3.015075
3,2002-03-07,Thursday,2002-09,2.220000,2.140000,0.080000,3.738318,3.738318,4.390244
4,2002-03-08,Friday,2002-09,2.270000,2.200000,0.070000,3.181818,2.252252,2.803738
...,...,...,...,...,...,...,...,...,...
7454,2022-07-31,Sunday,2022-30,286.666667,131.703333,154.963333,117.660905,-1.601831,0.180020
7455,2022-08-01,Monday,2022-31,282.000000,131.940000,150.060000,113.733515,-1.627907,0.179697
7456,2022-08-02,Tuesday,2022-31,291.000000,132.180000,158.820000,120.154335,3.191489,0.181901
7457,2022-08-03,Wednesday,2022-31,298.000000,132.410000,165.590000,125.058530,2.405498,0.174005


Continúo con la **variación** de la brecha

In [10]:
br_var = np.array([])
for i in table.index:
    if i == 0:
        var = 0
    else:
        var = table.loc[i,'brecha'] - table.loc[i-1,'brecha']
    br_var = np.append(br_var, var)
table['brecha_var'] = br_var
table.head()

,fecha,dia_semana,semana,dolar_b,dolar_o,brecha_s,brecha,db_var,do_var,brecha_var
0,2002-03-04,Monday,2002-09,2.12,2.01,0.11,5.472637,0.000000,0.000000,0.000000
1,2002-03-05,Tuesday,2002-09,2.12,1.99,0.13,6.532663,0.000000,-0.995025,1.060027
2,2002-03-06,Wednesday,2002-09,2.14,2.05,0.09,4.390244,0.943396,3.015075,-2.142419
3,2002-03-07,Thursday,2002-09,2.22,2.14,0.08,3.738318,3.738318,4.390244,-0.651926
4,2002-03-08,Friday,2002-09,2.27,2.20,0.07,3.181818,2.252252,2.803738,-0.556500


Junto los eventos a la tabla principal:

In [11]:
table = table.join(eventos.set_index('d'), on='fecha', how='left').rename(columns={'e': 'evento_p', 't':'evento_t'})
table.head(10)

,fecha,dia_semana,semana,dolar_b,dolar_o,brecha_s,brecha,db_var,do_var,brecha_var,evento_p,evento_t
0,2002-03-04,Monday,2002-09,2.120000,2.010000,0.11,5.472637,0.000000,0.000000,0.000000,NaN,NaN
1,2002-03-05,Tuesday,2002-09,2.120000,1.990000,0.13,6.532663,0.000000,-0.995025,1.060027,NaN,NaN
2,2002-03-06,Wednesday,2002-09,2.140000,2.050000,0.09,4.390244,0.943396,3.015075,-2.142419,NaN,NaN
3,2002-03-07,Thursday,2002-09,2.220000,2.140000,0.08,3.738318,3.738318,4.390244,-0.651926,NaN,NaN
4,2002-03-08,Friday,2002-09,2.270000,2.200000,0.07,3.181818,2.252252,2.803738,-0.556500,NaN,NaN
5,2002-03-09,Saturday,2002-09,2.286667,2.216667,0.07,3.157895,0.734214,0.757576,-0.023923,NaN,NaN
6,2002-03-10,Sunday,2002-09,2.303333,2.233333,0.07,3.134328,0.728863,0.751880,-0.023566,NaN,NaN
7,2002-03-11,Monday,2002-10,2.320000,2.250000,0.07,3.111111,0.723589,0.746269,-0.023217,NaN,NaN
8,2002-03-12,Tuesday,2002-10,2.430000,2.370000,0.06,2.531646,4.741379,5.333333,-0.579466,NaN,NaN
9,2002-03-13,Wednesday,2002-10,2.420000,2.330000,0.09,3.862661,-0.411523,-1.687764,1.331015,NaN,NaN


In [12]:
# Remplazo nulos
table['evento_p'] = table['evento_p'].fillna('') 
table['evento_t'] = table['evento_t'].fillna('')
table.head() 

,fecha,dia_semana,semana,dolar_b,dolar_o,brecha_s,brecha,db_var,do_var,brecha_var,evento_p,evento_t
0,2002-03-04,Monday,2002-09,2.12,2.01,0.11,5.472637,0.000000,0.000000,0.000000,,
1,2002-03-05,Tuesday,2002-09,2.12,1.99,0.13,6.532663,0.000000,-0.995025,1.060027,,
2,2002-03-06,Wednesday,2002-09,2.14,2.05,0.09,4.390244,0.943396,3.015075,-2.142419,,
3,2002-03-07,Thursday,2002-09,2.22,2.14,0.08,3.738318,3.738318,4.390244,-0.651926,,
4,2002-03-08,Friday,2002-09,2.27,2.20,0.07,3.181818,2.252252,2.803738,-0.556500,,


In [13]:
table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7468 entries, 0 to 7458
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   fecha       7468 non-null   datetime64[ns]
 1   dia_semana  7468 non-null   object        
 2   semana      7468 non-null   object        
 3   dolar_b     7468 non-null   float64       
 4   dolar_o     7468 non-null   float64       
 5   brecha_s    7468 non-null   float64       
 6   brecha      7468 non-null   float64       
 7   db_var      7468 non-null   float64       
 8   do_var      7468 non-null   float64       
 9   brecha_var  7468 non-null   float64       
 10  evento_p    7468 non-null   object        
 11  evento_t    7468 non-null   object        
dtypes: datetime64[ns](1), float64(7), object(4)
memory usage: 758.5+ KB


----
## Exportar tabla
Ya que tengo la tabla como prefiero, procedo a guardarla:

In [14]:
table.to_csv('my_tables/table_hist.csv', index= False)